### Prepare raw data

Download html files from site

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!find /content/drive/MyDrive/povar_archive/povar.ru/recipes/ -type f | wc -l

55104


In [ ]:
!wget -r -k -l 7 -p -E -nc --limit-rate=500k --random-wait --user-agent="Mozilla/5.0" -P /content/drive/MyDrive/povar_archive/ https://povar.ru/recipes/
# !rm -rf ./chroma_db

Выходные данные были обрезаны до нескольких последних строк (5000).
2025-04-25 00:37:57 (792 KB/s) - ‘/content/drive/MyDrive/povar_archive/povar.ru/recipes/morkovnaya_zapekanka_s_chernoslivom-82582.html’ saved [174475]

--2025-04-25 00:37:57--  https://povar.ru/recipes/zapekanka_iz_morkovi_s_farshem-81783.html
Reusing existing connection to povar.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/drive/MyDrive/povar_archive/povar.ru/recipes/zapekanka_iz_morkovi_s_farshem-81783.html’

povar.ru/recipes/za     [  <=>               ] 166.70K   789KB/s    in 0.2s    

2025-04-25 00:37:57 (789 KB/s) - ‘/content/drive/MyDrive/povar_archive/povar.ru/recipes/zapekanka_iz_morkovi_s_farshem-81783.html’ saved [170701]

--2025-04-25 00:37:57--  https://povar.ru/recipes/morkovnaya_zapekanka_s_kuragoi-81441.html
Reusing existing connection to povar.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to:

In [ ]:
!cp -r /content/povar.ru/ /content/drive/MyDrive/

In [ ]:
!cd /content/drive/MyDrive/

In [ ]:
!ls

drive  sample_data


In [ ]:
!ls -l /content/drive/MyDrive/povar.ru/

ls: cannot access '/content/drive/MyDrive/povar.ru/': No such file or directory


In [ ]:
# !apt-get update
# !apt-get install unzip
# !unzip povar_recipes_1.zip
# !unzip povar_recipes_2.zip
# !unzip povar_recipes_3.zip

In [2]:
import os
import random
import shutil

# Пути
source_dir = '/content/drive/MyDrive/povar_archive/povar.ru/recipes/'  # Исходная папка
target_dir = '/content/drive/MyDrive/demo_500_recipes/'  # Новая папка для 500 рецептов

# Создаем целевую папку
os.makedirs(target_dir, exist_ok=True)

In [4]:
import glob

# Используем glob для получения файлов
all_files = glob.glob(os.path.join(source_dir, '*.html'))
sampled_files = random.sample(all_files, 500)

# Копирование с progress bar
from tqdm import tqdm
for file in tqdm(sampled_files, desc="Копирование файлов"):
    shutil.copy(file, target_dir)

Копирование файлов: 100%|██████████| 500/500 [04:41<00:00,  1.78it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/llm_kaggle/llm_kaggle

/content/drive/MyDrive/llm_kaggle/llm_kaggle


In [5]:
!ls

chroma_db  experiments		logs		     requirements.txt
config	   init_environment.sh	Recipe_final.pickle  scripts
dataset    log.pickle		recipes_


In [6]:
!pip install python-Levenshtein  # Ускорит работу в 4-10 раз

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.2 MB/s eta 0:00:00


In [7]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 81.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.8.5) =================
ℹ spaCy installation: /usr/local/lib/python3.11/dist-packages/spacy

NAME              SPACY            VERSION                            
ru_core_news_sm   >=3.8.0,<3.9.0   3.8.0   ✔
en_core_web_sm    >=3.8.0,<3.9.0   3.8.0   ✔



In [10]:
# Установка Ollama с явным указанием пути к CUDA
!curl -fsSL https://ollama.com/install.sh | OLLAMA_CUDA=1 sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [11]:
# Запуск с GPU (без флага NO_CUDA)
!ollama serve > /tmp/ollama.log 2>&1 &
!sleep 5  # Даем время на инициализацию

# Проверка использования GPU
!nvidia-smi  # Должна отображаться загрузка GPU процессом "ollama"

Fri Apr 25 13:39:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
!curl http://localhost:11434

Ollama is running

In [13]:
!ps aux | grep ollama

root        4267  3.1  0.3 6457760 41280 ?       Sl   13:39   0:00 ollama serve
root        4301  0.0  0.0   7376  3436 ?        S    13:39   0:00 /bin/bash -c ps aux | grep ollama
root        4303  0.0  0.0   6484  2232 ?        R    13:39   0:00 grep ollama


In [14]:
!ollama pull mistral

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest 
pulling ff82381e2bea:   0% ▕▏ 1.1 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   1% ▕▏  61 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   3% ▕▏ 113 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   3% ▕▏ 136 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   4% ▕▏ 181 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   5% ▕▏ 218 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   6% ▕▏ 248 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   7% ▕▏ 298 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   8% ▕▏ 325 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   9% ▕▏ 377 MB/4.1 GB                  p

### Make DataSet

In [15]:
!ls

chroma_db  experiments		logs		     requirements.txt
config	   init_environment.sh	Recipe_final.pickle  scripts
dataset    log.pickle		recipes_


In [16]:
!python3 scripts/pipeline.py

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Loaded 0 recipes
Processing Recipes ...
0it [00:00, ?it/s]
INFO:config.helpers:Filtered 0/0 recipes
Making Vector Store ...
Using device: cuda
Traceback (most recent call last):
  File "/content/drive/MyDrive/llm_kaggle/llm_kaggle/scripts/pipeline.py", line 91, in <module>
    main()
  File "/content/drive/MyDrive/llm_kaggle/llm_kaggle/scripts/pipeline.py", line 63, in main
    vector_store, tokenizer = create_db(recipes_list)
                              ^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/drive/MyDrive/llm_kaggle/llm_kaggle/scripts/pipeline.py", line 40, in create_db
    vector_store, tokenizer = build_chroma_db(
                              ^^^^^^^^^^^^^^^^
TypeError: build_chroma_db() got an unexpected keyword argument 'embedding_model'


In [ ]:
%load_ext autoreload
%autoreload
from config.helpers import make_data_set, read_pkl, save_pkl, make_recipes,filter_recipe

%load_ext autoreload
%autoreload
from config.dish import Recipe, RecipesProject

MAIN_DIR = "/content/drive/MyDrive/llm_kaggle/llm_kaggle/dataset/"
FILE_NAME = "recipes_"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os
from pathlib import Path

# Убедитесь, что MAIN_DIR указан правильно
MAIN_DIR = "/content/drive/MyDrive/llm_kaggle/llm_kaggle/dataset/"
FILE_NAME = os.path.join(MAIN_DIR, "recipes_")  # Полный путь

# Проверка существования файлов
for i in range(1, 4):
    file_path = f"{FILE_NAME}{i}.pickle"
    print(f"Файл {file_path} существует: {os.path.exists(file_path)}")

Файл /content/drive/MyDrive/llm_kaggle/llm_kaggle/dataset/recipes_1.pickle существует: True
Файл /content/drive/MyDrive/llm_kaggle/llm_kaggle/dataset/recipes_2.pickle существует: True
Файл /content/drive/MyDrive/llm_kaggle/llm_kaggle/dataset/recipes_3.pickle существует: True


In [ ]:
povar_recipes = make_data_set(MAIN_DIR)
save_pkl(povar_recipes, FILE_NAME)

povar_recipes = []
for i in range(1, 4):
    _povar_recipes = read_pkl(FILE_NAME + str(i) + ".pickle")
    povar_recipes.extend(_povar_recipes)
len(povar_recipes)

0it [00:00, ?it/s]


66349

In [ ]:
recipes_list = make_recipes(povar_recipes)

100%|██████████| 66349/66349 [00:07<00:00, 8480.05it/s] 


In [ ]:
recipes_list = filter_recipe(recipes_list, max_steps = 10, max_min=120, min_rating=4, min_votes=2)
save_pkl(recipes_list, "./Recipe_final.pickle")

In [ ]:
recipes_list[13].document.page_content

'Пирожки на воде мука, вода, дрожжи сухие, масло растительное, сахар, сольСмешайте 300 мл теплой воды, 1 ст.л. дрожжей сухих, 2 ст.л. сахара, 3 ст.л. муки., Поставьте разведенные дрожжи на 15 минут в теплое место., Добавьте масло и соль., Замесите тесто, постепенно добавляя муку  до такого состояния, когда оно перестанет приставать к рукам. Поставьте тесто в теплое место на 15 минут., Тесто разделите на лепешки, положите начинку и уложите пирожки на противень., Тесто разделите на лепешки, положите начинку и слепите пирожки., Пирожки выпекайте в разогретом масле до золотистого цвета.'

### Create Chroma DB

In [ ]:
# 1. Создаем рабочую директорию
os.makedirs("/content/drive/MyDrive/llm_kaggle/llm_kaggle/chroma_db", exist_ok=True)

In [ ]:
# Chroma DB
%load_ext autoreload
%autoreload
from config.vector_db import build_chroma_db, request_chroma_db
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
vector_store, tokenizer = build_chroma_db(
    recipes_list,
    model_name="ai-forever/sbert_large_nlu_ru", #"nomic-ai/nomic-embed-text-v1.5", #"IlyaGusev/saiga_yandexgpt_8b"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Create Knowledgr Graph

In [ ]:
### Knowledge graph
%load_ext autoreload
%autoreload
from config.food_graph import make_tags_list, lemmatize_tags, build_knowledge_graph\
    , lemmatize, lemmatize_sentance, make_one_word_tags_list, enreach_query_with_relative_tags

tags = make_tags_list(recipes_list)

lemmatize_tags(tags)

one_word_tags = make_one_word_tags_list(tags)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
Graph = build_knowledge_graph(recipes_list, tags)

In [ ]:
rp = RecipesProject(
    recipes=recipes_list,
    knowledgeGraph=Graph,
    tags=tags,
    vectorStore=vector_store,
    oneWordTags=one_word_tags
)
rp.add_resipes_list(recipes_list)
rp.add_knowledge_graph(G)
rp.add_tags(tags)
rp.add_one_word_tags(one_word_tags)

NameError: name 'RecipesProject' is not defined

### Create LLM

In [ ]:
%load_ext autoreload
%autoreload
from config.model_loader import build_model, llm_invoke
llm = build_model()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Testing

In [ ]:
requests = [
    # Быстрые и простые блюда
    ["Как быстро приготовить ужин из минимальных ингредиентов?"],
    ["Простой рецепт завтрака за 10 минут"],
    ["Что можно быстро сделать из яиц и овощей?"],

    # Специфические техники приготовления
    ["Рецепт стейка средней прожарки в духовке"],
    ["Как правильно варить пасту аль денте?"],
    ["Техника приготовления воздушного безе"],

    # Международная кухня
    ["Аутентичный рецепт пад тай с курицей"],
    ["Как приготовить рамен как в Японии?"],
    ["Традиционный французский луковый суп"],

    # Диетические/специальные запросы
    ["Низкоуглеводный рецепт с курицей"],
    ["Веганский вариант бешамель"],
    ["Безглютеновая выпечка с миндальной мукой"],

    # Сезонные блюда
    ["Теплый салат для осеннего ужина"],
    ["Освежающий летний десерт без выпечки"],
    ["Зимний согревающий суп"],

    # Кухонные лайфхаки
    ["Как реанимировать пересоленный суп?"],
    ["Способы нарезки лука без слез"],
    ["Как сделать идеально пушистый рис"],

    # Нестандартные комбинации
    ["Что приготовить из бананов и курицы?"],
    ["Необычный десерт из авокадо и шоколада"],
    ["Салат с клубникой и сыром фета"],

    # Детское питание
    ["Полезные блинчики для ребенка 1 года"],
    ["Как сделать овощное пюре вкусным для детей"],

    # Алкогольные напитки
    ["Классический рецепт мохито"],
    ["Как сделать домашний лимонад с мятой"],

    # Кулинарные эксперименты
    ["Рецепт кофе с кардамоном и корицей"],
    ["Как замариновать мясо в кока-коле"],
    ["Необычный способ приготовления картофеля"]
]

### Queries

In [ ]:
recipes_dict = {r.id: r for r in rp.recipes}

In [ ]:
import random
import logging

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

def final_query(query, n=3):
    query_clean = query.lower().replace("рецепт", "").replace("кухня", "")
    results_kg = rp.invoke(query_clean, verbose=False)
    if len(results_kg[0]) == 0:
        results_kg = results_kg[1]
    else:
        results_kg = results_kg[0]
    id_kg = [str(id) for id in results_kg]

    results_db = request_chroma_db(vector_store, tokenizer, query, device, k=100)

    id_db = []
    for r in results_db:
        if hasattr(r, 'metadata') and isinstance(r.metadata, dict) and 'id' in r.metadata:
            id_db.append(r.metadata['id'])
        else:
            logger.warning(f"Отсутствует 'id' в метаданных для результата: {r}")
            continue

    res_final = []
    for r in results_db:
        if hasattr(r, 'metadata') and isinstance(r.metadata, dict) and 'id' in r.metadata:
            if r.metadata['id'] in id_kg:
                res_final.append(r.metadata['id'])

    if len(res_final) < n:
        kg_rest = [id for id in id_kg if id not in id_db]
        res_final.extend(random.sample(kg_rest, k=min(n - len(res_final), len(kg_rest))))

    if len(res_final) < n:
        db_rest = [id for id in id_db if id not in res_final]
        res_final.extend(db_rest[:n - len(res_final)])

    if not id_db:
        logger.warning(f"Не удалось извлечь ID из Chroma DB для запроса: {query}")

    return id_kg[:n], id_db[:n], res_final[:n]

In [ ]:
%load_ext autoreload
%autoreload
from scripts.prompt_engine import create_chat_history, prompt, formatted_chat_history, formatted_rag
chat_history = create_chat_history()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
for query in tqdm(requests[:10]):
    log.append([])
    chat_history = create_chat_history()
    for question in query:
        start_time = time.time()
        kg, db, final = final_query(question)

        # Защищенное получение рецептов
        valid_recipes = []
        missing_ids = []
        for r in final:
            if r in recipes_dict:
                valid_recipes.append(recipes_dict[r].display)
            else:
                missing_ids.append(r)

        if missing_ids:
            print(f"Предупреждение: отсутствуют рецепты с ID: {missing_ids}")

        format_chat_history = formatted_chat_history(chat_history)
        format_rag = formatted_rag(valid_recipes) if valid_recipes else "Нет данных"

        question_input = {
            "chat_history": format_chat_history,
            "user_input": question,
            "retrieved_recipes": format_rag
        }

        final_prompt = prompt.format(**question_input)
        llm_prompt_result = llm_invoke(llm, final_prompt)
        llm_prompt_time = time.time() - start_time

        start_time = time.time()
        llm_question_result = llm_invoke(llm, question)
        llm_question_time = time.time() - start_time

        chat_history.add_user_message(question)
        chat_history.add_ai_message(llm_prompt_result)

        log[-1].append({
            'kg': kg,
            'db': db,
            'missing_ids': missing_ids,
            'llm_question': llm_question_result,
            'time_question': llm_question_time,
            'llm_prompt': llm_prompt_result,
            'time_prompt': llm_prompt_time,
        })

  0%|          | 0/10 [00:00<?, ?it/s]

Result 0: metadata={'id': '34624', 'source': 'recipe'}
Result 1: metadata={'id': '33223', 'source': 'recipe'}
Result 2: metadata={'id': '24712', 'source': 'recipe'}
Result 3: metadata={'id': '34974', 'source': 'recipe'}
Result 4: metadata={'id': '34837', 'source': 'recipe'}
Result 5: metadata={'id': '8816', 'source': 'recipe'}
Result 6: metadata={'id': '20943', 'source': 'recipe'}
Result 7: metadata={'id': '21052', 'source': 'recipe'}
Result 8: metadata={'id': '20041', 'source': 'recipe'}
Result 9: metadata={'id': '34801', 'source': 'recipe'}
Result 10: metadata={'id': '16080', 'source': 'recipe'}
Result 11: metadata={'id': '20154', 'source': 'recipe'}
Result 12: metadata={'id': '30934', 'source': 'recipe'}
Result 13: metadata={'id': '34777', 'source': 'recipe'}
Result 14: metadata={'id': '29986', 'source': 'recipe'}
Result 15: metadata={'id': '32513', 'source': 'recipe'}
Result 16: metadata={'id': '29117', 'source': 'recipe'}
Result 17: metadata={'id': '17427', 'source': 'recipe'}
Res

 10%|█         | 1/10 [00:34<05:14, 34.98s/it]

Result 0: metadata={'id': '7118', 'source': 'recipe'}
Result 1: metadata={'id': '26850', 'source': 'recipe'}
Result 2: metadata={'id': '20041', 'source': 'recipe'}
Result 3: metadata={'id': '26010', 'source': 'recipe'}
Result 4: metadata={'id': '17784', 'source': 'recipe'}
Result 5: metadata={'id': '24559', 'source': 'recipe'}
Result 6: metadata={'id': '8816', 'source': 'recipe'}
Result 7: metadata={'id': '26921', 'source': 'recipe'}
Result 8: metadata={'id': '34624', 'source': 'recipe'}
Result 9: metadata={'id': '28490', 'source': 'recipe'}
Result 10: metadata={'id': '27255', 'source': 'recipe'}
Result 11: metadata={'id': '12038', 'source': 'recipe'}
Result 12: metadata={'id': '19614', 'source': 'recipe'}
Result 13: metadata={'id': '20692', 'source': 'recipe'}
Result 14: metadata={'id': '8908', 'source': 'recipe'}
Result 15: metadata={'id': '33176', 'source': 'recipe'}
Result 16: metadata={'id': '18968', 'source': 'recipe'}
Result 17: metadata={'id': '20190', 'source': 'recipe'}
Resul

 20%|██        | 2/10 [01:04<04:12, 31.51s/it]

...

 80%|████████  | 8/10 [04:45<01:15, 37.75s/it]

Result 0: metadata={'id': '12038', 'source': 'recipe'}
Result 1: metadata={'id': '8816', 'source': 'recipe'}
Result 2: metadata={'id': '9131', 'source': 'recipe'}
Result 3: metadata={'id': '34624', 'source': 'recipe'}
Result 4: metadata={'id': '33282', 'source': 'recipe'}
Result 5: metadata={'id': '34974', 'source': 'recipe'}
Result 6: metadata={'id': '17599', 'source': 'recipe'}
Result 7: metadata={'id': '34801', 'source': 'recipe'}
Result 8: metadata={'id': '34837', 'source': 'recipe'}
Result 9: metadata={'id': '24712', 'source': 'recipe'}
Result 10: metadata={'id': '218', 'source': 'recipe'}
Result 11: metadata={'id': '16080', 'source': 'recipe'}
Result 12: metadata={'id': '35203', 'source': 'recipe'}
Result 13: metadata={'id': '8822', 'source': 'recipe'}
Result 14: metadata={'id': '12059', 'source': 'recipe'}
Result 15: metadata={'id': '29117', 'source': 'recipe'}
Result 16: metadata={'id': '21052', 'source': 'recipe'}
Result 17: metadata={'id': '17444', 'source': 'recipe'}
Result 

 90%|█████████ | 9/10 [05:05<00:32, 32.28s/it]

Result 0: metadata={'id': '17784', 'source': 'recipe'}
Result 1: metadata={'id': '34837', 'source': 'recipe'}
Result 2: metadata={'id': '35203', 'source': 'recipe'}
Result 3: metadata={'id': '17444', 'source': 'recipe'}
Result 4: metadata={'id': '34801', 'source': 'recipe'}
Result 5: metadata={'id': '12038', 'source': 'recipe'}
Result 6: metadata={'id': '8816', 'source': 'recipe'}
Result 7: metadata={'id': '17427', 'source': 'recipe'}
Result 8: metadata={'id': '9131', 'source': 'recipe'}
Result 9: metadata={'id': '34974', 'source': 'recipe'}
Result 10: metadata={'id': '34624', 'source': 'recipe'}
Result 11: metadata={'id': '9943', 'source': 'recipe'}
Result 12: metadata={'id': '17599', 'source': 'recipe'}
Result 13: metadata={'id': '14914', 'source': 'recipe'}
Result 14: metadata={'id': '24712', 'source': 'recipe'}
Result 15: metadata={'id': '33282', 'source': 'recipe'}
Result 16: metadata={'id': '11904', 'source': 'recipe'}
Result 17: metadata={'id': '20692', 'source': 'recipe'}
Resul

100%|██████████| 10/10 [05:34<00:00, 33.42s/it]


In [ ]:
for query, ll in zip(requests, log):
    for q, l in zip(query, ll):
        try:
            # Проверяем, есть ли ID в словаре (KG)
            kg_id = l['kg'][0] if l['kg'] else None
            kg = recipes_dict[kg_id].display if kg_id and kg_id in recipes_dict else "Рецепт не найден в KG"

            # Проверяем, есть ли ID в словаре (DB)
            db_id = l['db'][0] if l['db'] else None
            db = recipes_dict[db_id].display if db_id and db_id in recipes_dict else "Рецепт не найден в DB"

            print(f"\n\n**Question**: {q}\n\n\t**KG**: {kg}\n\n\t**DB**: {db}\n\n\t**LLM_Prompt**: {l['llm_prompt']}\n\n\t**LLM_Query**: {l['llm_question']}")

        except KeyError as e:
            print(f"\n\n Ошибка: ID {e} отсутствует в recipes_dict для вопроса: '{q}'")
            print(f"Проверьте данные для запроса: KG={l['kg']}, DB={l['db']}")




**Question**: Простой рецепт завтрака за 10 минут

	**KG**: Салат из кальмаров и мидий с чипсами "Жемчужина моря"

Количество порций: 3.0.
Ингридиенты
	сыр моцарелла: 100 грамм.
	чипсы: 1 банка.
	соль: 1 по вкусу.
	помидоры черри: 20 штук.
	кунжут: 1 по вкусу.
	салат айсберг: 1 штука.
	перец черный: 1 по вкусу.
	мидии в масле или рассоле: 1 банка.
	щупальца кальмара в масле или рассоле: 1 банка.
	масло оливковое ( заправки): 4 ст. ложки.

Способ приготовления
	1. Приготовьте все продукты для салата..
	2. Для начала нам необходимо нарезать помидоры черри на 4 части, а кусочки сыра моцарелла на 2 части..
	3. Салат очищаем от верхних листов и нарезаем поперек полосками 1 см шириной. При этом оставляем большой хороший лист для украшения..
	4. Морепродукты в нашем случае уже готовы, что существенно экономит нам время, поэтому их остается только открыть. Поскольку мы используем кальмары и мидии в масле, то при выкладывании старайтесь чтобы масло с них стекло..
	5. Салат мы будем собирать п

In [ ]:
import pickle

with open("log.pickle", 'wb') as handle:
        pickle.dump(log, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
test_queries = [
    # Итальянская кухня
    ["Как правильно приготовить пасту карбонара?", "Какой сыр лучше использовать для карбонары?"],
    ["Рецепт классического соуса болоньезе", "Чем заменить говяжий фарш в болоньезе?"],

    # Детское питание
    ["Простое детское блюдо с курицей и овощами"],
    ["Что приготовить ребенку на ужин из индейки?"],

    # Выпечка
    ["Быстрый рецепт шарлотки с яблоками"],
    ["Как испечь пирог с яблоками за 30 минут?"],

    # Овощные блюда
    ["Рецепт оладий из кабачков"],
    ["Как приготовить рататуй в духовке?"],

    # Рыбные блюда
    ["Рецепт запеченной рыбы с картошкой"],
    ["Как пожарить рыбу с хрустящей корочкой?"],

    # Быстрые решения
    ["Как быстро накормить семью за 15 минут?"],
    ["Экспресс-рецепты когда совсем нет времени"],

    # Фруктовые десерты
    ["Рецепт фруктового салата без бананов"],
    ["Какие фрукты сочетаются в летнем салате?"],

    # Альтернативные варианты
    ["Чем заменить майонез в салате?"],
    ["Как сделать вегетарианский вариант оливье?"]
]

In [ ]:
chat_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='Низкоуглеводный рецепт с курицей', additional_kwargs={}, response_metadata={}), AIMessage(content=' **Название рецепта:** Курица с овощами "Рецепт ленивой хозяйки"\n\n   **Количество порций:** 4\n\n   **Список ингредиентов:**\n   - Куриное бедро: 600-800 грамм\n   - Капуста белокочанная: 600 грамм\n   - Лук репчатый: 1 штука\n   - Морковь: 1 штука\n   - Перец болгарский: 1 штука\n   - Чеснок: 1-2 зубчика\n   - Соль: по вкусу\n   - Специи: по вкусу\n\n   **Способ приготовления:**\n   1. Подготовьте все необходимые ингредиенты.\n   2. Куриные бедра помойте, посолите и поперчите по вкусу. По желанию добавьте любимые специи.\n   3. Овощи хорошо помойте и очистите. Капусту нашинкуйте. Если капуста не молодая, то немного помните руками. Лук нарежьте полукольцами, морковь натрите на крупной терке, перец нарежьте соломкой, чеснок мелко порубите.\n   4. Переложите овощи в пакет для запекания, добавьте по вкусу соль, закрутите хвостик па

In [ ]:
prompt

PromptTemplate(input_variables=['chat_history', 'retrieved_recipes', 'user_input'], input_types={}, partial_variables={}, template='\nВы — опытный кулинарный помощник. Ваша цель — давать полезные, креативные и простые в использовании советы по приготовлению пищи.\nУ вас есть доступ к большой базе рецептов, хранящейся в ChromaDB, которая позволяет вам извлекать соответствующие рецепты и предлагать\nперсонализированные идеи приготовления пищи на основе предпочтений пользователя, доступных ингредиентов, диетических ограничений и стиля приготовления.\n\nИспользуйте историю чата, чтобы сохранять контекст и избегать повторений. Если пользователь упомянул определенные ингредиенты,\nпредпочтения или антипатии ранее в разговоре, учтите это при предоставлении совета.\n\nИспользуйте выбранные рецепты из ChromaDB в качестве руководства для создания индивидуальных предложений. Вы можете комбинировать элементы из разных рецептов или адаптировать их на основе предпочтений пользователя.\n\nЧетко форма

In [ ]:
for query in tqdm(requests[:10]):
    chat_history = create_chat_history()
    for question in query:
        try:
            start_time = time.time()

            # 1. Получаем результаты запроса
            kg, db, final_ids = final_query(question)

            # 2. Безопасно извлекаем рецепты
            valid_recipes = []
            missing_ids = []

            for recipe_id in final_ids:
                if str(recipe_id) in recipes_dict:  # Проверяем как строку
                    valid_recipes.append(recipes_dict[str(recipe_id)].display)
                else:
                    missing_ids.append(recipe_id)

            if missing_ids:
                print(f"Предупреждение: отсутствуют рецепты с ID: {missing_ids}")

            # 3. Форматируем данные
            format_chat_history = formatted_chat_history(chat_history)
            format_rag = formatted_rag(valid_recipes) if valid_recipes else "Рецепты не найдены"

            # 4. Собираем входные данные
            question_input = {
                "chat_history": format_chat_history,
                "user_input": question,
                "retrieved_recipes": format_rag
            }

            # 5. Генерируем и выводим промпт
            final_prompt = prompt.format(**question_input)
            print(final_prompt)

        except Exception as e:
            print(f"Ошибка при обработке вопроса '{question}': {str(e)}")
            continue

 10%|█         | 1/10 [00:00<00:01,  6.06it/s]

Result 0: metadata={'id': '34624', 'source': 'recipe'}
Result 1: metadata={'id': '33223', 'source': 'recipe'}
Result 2: metadata={'id': '24712', 'source': 'recipe'}
Result 3: metadata={'id': '34974', 'source': 'recipe'}
Result 4: metadata={'id': '34837', 'source': 'recipe'}
Result 5: metadata={'id': '8816', 'source': 'recipe'}
Result 6: metadata={'id': '20943', 'source': 'recipe'}
Result 7: metadata={'id': '21052', 'source': 'recipe'}
Result 8: metadata={'id': '20041', 'source': 'recipe'}
Result 9: metadata={'id': '34801', 'source': 'recipe'}
Result 10: metadata={'id': '16080', 'source': 'recipe'}
Result 11: metadata={'id': '20154', 'source': 'recipe'}
Result 12: metadata={'id': '30934', 'source': 'recipe'}
Result 13: metadata={'id': '34777', 'source': 'recipe'}
Result 14: metadata={'id': '29986', 'source': 'recipe'}
Result 15: metadata={'id': '32513', 'source': 'recipe'}
Result 16: metadata={'id': '29117', 'source': 'recipe'}
Result 17: metadata={'id': '17427', 'source': 'recipe'}
Res

 40%|████      | 4/10 [00:00<00:00,  8.15it/s]

Result 0: metadata={'id': '34837', 'source': 'recipe'}
Result 1: metadata={'id': '8816', 'source': 'recipe'}
Result 2: metadata={'id': '34801', 'source': 'recipe'}
Result 3: metadata={'id': '34624', 'source': 'recipe'}
Result 4: metadata={'id': '17784', 'source': 'recipe'}
Result 5: metadata={'id': '33223', 'source': 'recipe'}
Result 6: metadata={'id': '16080', 'source': 'recipe'}
Result 7: metadata={'id': '17427', 'source': 'recipe'}
Result 8: metadata={'id': '8822', 'source': 'recipe'}
Result 9: metadata={'id': '17444', 'source': 'recipe'}
Result 10: metadata={'id': '24712', 'source': 'recipe'}
Result 11: metadata={'id': '34974', 'source': 'recipe'}
Result 12: metadata={'id': '20379', 'source': 'recipe'}
Result 13: metadata={'id': '35203', 'source': 'recipe'}
Result 14: metadata={'id': '26921', 'source': 'recipe'}
Result 15: metadata={'id': '9131', 'source': 'recipe'}
Result 16: metadata={'id': '34777', 'source': 'recipe'}
Result 17: metadata={'id': '24711', 'source': 'recipe'}
Resul

 60%|██████    | 6/10 [00:00<00:00,  7.55it/s]

Result 0: metadata={'id': '20557', 'source': 'recipe'}
Result 1: metadata={'id': '8816', 'source': 'recipe'}
Result 2: metadata={'id': '26921', 'source': 'recipe'}
Result 3: metadata={'id': '8852', 'source': 'recipe'}
Result 4: metadata={'id': '9631', 'source': 'recipe'}
Result 5: metadata={'id': '29986', 'source': 'recipe'}
Result 6: metadata={'id': '24712', 'source': 'recipe'}
Result 7: metadata={'id': '22765', 'source': 'recipe'}
Result 8: metadata={'id': '25460', 'source': 'recipe'}
Result 9: metadata={'id': '32592', 'source': 'recipe'}
Result 10: metadata={'id': '32513', 'source': 'recipe'}
Result 11: metadata={'id': '20154', 'source': 'recipe'}
Result 12: metadata={'id': '8353', 'source': 'recipe'}
Result 13: metadata={'id': '21329', 'source': 'recipe'}
Result 14: metadata={'id': '7993', 'source': 'recipe'}
Result 15: metadata={'id': '1595', 'source': 'recipe'}
Result 16: metadata={'id': '31621', 'source': 'recipe'}
Result 17: metadata={'id': '9506', 'source': 'recipe'}
Result 18

 80%|████████  | 8/10 [00:01<00:00,  7.27it/s]

Result 0: metadata={'id': '7118', 'source': 'recipe'}
Result 1: metadata={'id': '17784', 'source': 'recipe'}
Result 2: metadata={'id': '34837', 'source': 'recipe'}
Result 3: metadata={'id': '12038', 'source': 'recipe'}
Result 4: metadata={'id': '35203', 'source': 'recipe'}
Result 5: metadata={'id': '17444', 'source': 'recipe'}
Result 6: metadata={'id': '34801', 'source': 'recipe'}
Result 7: metadata={'id': '33176', 'source': 'recipe'}
Result 8: metadata={'id': '9131', 'source': 'recipe'}
Result 9: metadata={'id': '17427', 'source': 'recipe'}
Result 10: metadata={'id': '20692', 'source': 'recipe'}
Result 11: metadata={'id': '8816', 'source': 'recipe'}
Result 12: metadata={'id': '33175', 'source': 'recipe'}
Result 13: metadata={'id': '9943', 'source': 'recipe'}
Result 14: metadata={'id': '17599', 'source': 'recipe'}
Result 15: metadata={'id': '34624', 'source': 'recipe'}
Result 16: metadata={'id': '33282', 'source': 'recipe'}
Result 17: metadata={'id': '20758', 'source': 'recipe'}
Result

100%|██████████| 10/10 [00:01<00:00,  6.87it/s]

Result 0: metadata={'id': '12038', 'source': 'recipe'}
Result 1: metadata={'id': '8816', 'source': 'recipe'}
Result 2: metadata={'id': '9131', 'source': 'recipe'}
Result 3: metadata={'id': '34624', 'source': 'recipe'}
Result 4: metadata={'id': '33282', 'source': 'recipe'}
Result 5: metadata={'id': '34974', 'source': 'recipe'}
Result 6: metadata={'id': '17599', 'source': 'recipe'}
Result 7: metadata={'id': '34801', 'source': 'recipe'}
Result 8: metadata={'id': '34837', 'source': 'recipe'}
Result 9: metadata={'id': '24712', 'source': 'recipe'}
Result 10: metadata={'id': '218', 'source': 'recipe'}
Result 11: metadata={'id': '16080', 'source': 'recipe'}
Result 12: metadata={'id': '35203', 'source': 'recipe'}
Result 13: metadata={'id': '8822', 'source': 'recipe'}
Result 14: metadata={'id': '12059', 'source': 'recipe'}
Result 15: metadata={'id': '29117', 'source': 'recipe'}
Result 16: metadata={'id': '21052', 'source': 'recipe'}
Result 17: metadata={'id': '17444', 'source': 'recipe'}
Result 

In [ ]:
ground_truth = ["""
### 🎃 **Суп-пюре из тыквы с имбирем**

#### **Ингредиенты**
- Тыква — 500 г
- Морковь — 1 шт.
- Лук — 1 шт.
- Имбирь (свежий) — 1 см
- Сливки (20%) — 200 мл
- Оливковое масло — 2 ст. ложки
- Соль, перец — по вкусу

#### **Инструкция**
1. Очистите тыкву, морковь и лук, нарежьте кубиками.
2. Обжарьте лук и морковь на оливковом масле 5 минут.
3. Добавьте тыкву и имбирь, залейте водой (500 мл), варите 20 минут.
4. Погружным блендером измельчите суп до однородности.
5. Влейте сливки, посолите и поперчите. Подавайте с гренками.

#### **Советы**
✅ Для пикантности добавьте щепотку мускатного ореха.
✅ Можно заменить сливки на кокосовое молоко для веганского варианта.
""",
]

In [ ]:
from evaluate import load

bertscore = load("bertscore")
predictions = ["""1. **Название**: Суп-пюре из тыквы с морковью
   - Ингредиенты:
     - Тыква (500 г), морковь (1 шт.), лук (1 шт.), сливки (200 мл), соль.
   - Инструкция:
     - Варите овощи 20 минут, затем взбейте блендером. Добавьте сливки.
   - Время: 30 мин.""", "general kenobi"]
references = ["""
1. **Суп-пюре "Тепло души"**
   - Ингредиенты: Тыква, перец, морковь, имбирь, сельдерей.
   - Способ: Овощи обжарить, сварить, пюрировать.
   - Время: 1 час.

2. **Тыквенная каша с рисом**
   - Ингредиенты: Тыква, молоко, рис.
   - Способ: Варить тыкву с рисом до мягкости.
   - Время: 1 час.

""", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="ru")

In [ ]:
results

{'precision': [0.7065095901489258, 0.9999998807907104],
 'recall': [0.7310776710510254, 0.9999998807907104],
 'f1': [0.7185837030410767, 0.9999998807907104],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.51.3)'}